----
# Data Cleaning
----

### Notebook Overview

In this notebook, I will perform data cleaning and prepare the scraped dataset for exploratory data analysis (EDA). The key steps involved include::

- **Removing Duplicates:** Dropping duplicated rows to ensure accurate analysis.

- **Handling Missing Values:** Addressing missing values within the dataset to ensure high data quality.

- **Feature Extraction:** Enhancing the dataset by extracting relevant features from product descriptions.

## Set Up
---

In [144]:
import numpy as np
import pandas as pd
import re
import matplotlib


## Data Loading
----

In [145]:
df = pd.read_csv('../../data/scraped_data.csv', index_col = 0)


## Utility Functions

In [146]:
def df_check(df):
    '''
    Outputs quality measures for dataframes

    Paramters
    ---------
    df: DataFrame for quality check

    Returns
    -------
    Statements with data quality info such as shape, duplicated values, missing values
    '''
    
    shape = df.shape
    # Calling sum twice - first sum returns column level results second sum to retrun total null values in all columns
    null_vals = df.isna().sum().sum()
    duplicated_rows = df.duplicated().sum()
    duplicated_cols = df.columns.duplicated().sum()

    print (
    f"""
    Data Quality Checks:
    --------------------------------------------
    No. of rows: {shape[0]}
    No. of columns: {shape[1]}
    No. of missing values: {null_vals}
    No. of duplicated rows: {duplicated_rows}
    No. of duplicated columns: {duplicated_cols}
    """
)
    


In [147]:
def search_description(description, regexp):
    '''
    Outputs binary value

    Paramters
    ---------
    description: string of product description
    regexp: regular expression

    Returns
    -------
    1 if regexp is present in description, 0 if not
    '''
    if re.search(regexp, description.lower()):
        return 1
    else:
        return 0

In [148]:
def get_colour(description):
    '''
    Outputs colour in product description

    Paramters
    ---------
    description: string of product description

    Returns
    -------
    Colour mentioned in the description
    '''
    
    # Using matplotlib to get list of colours (instead of manually creating a list)    
    colour_names = matplotlib.colors.CSS4_COLORS.keys()

    # Looping through the list of colours to see if any of the colours are in the product description 
    for colour in colour_names:
        if re.search(rf'\b{colour}\b', description):
            return colour
        else:
            # if first colour in the list is not found try the next
            continue
    
    # Cases where no colour in the the colour_list is found in description
    return 'Not Specified'
    

In [149]:
def get_battery_life(description):
    '''
    Outputs battery life listed in product description

    Paramters
    ---------
    description: string of product description

    Returns
    -------
    Battery life in hours
    '''
        
    regexp = r'(\b[1-9]\d*)\s*(battery|batteries|hours?|hrs?|h)'
    if re.search(regexp,description):
        # using .group to only get the int part of the regexp
        return re.search(regexp,description).group(1)    
        
    else:
        return 'Not Specified'

## Preliminary Checks

In [150]:
df_check(df)


    Data Quality Checks:
    --------------------------------------------
    No. of rows: 1500
    No. of columns: 5
    No. of missing values: 687
    No. of duplicated rows: 468
    No. of duplicated columns: 0
    


In [151]:
df.info() # Checking data types

<class 'pandas.core.frame.DataFrame'>
Index: 1500 entries, 0 to 1499
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Product ID   1500 non-null   object
 1   Description  1500 non-null   object
 2   Price        1500 non-null   object
 3   Rating       813 non-null    object
 4   Is Prime     1500 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 70.3+ KB


### Dealing the with the duplicates

In [152]:
df[df.duplicated(keep=False)].sort_values(by = 'Product ID', ascending=False) # quick 4 eyes check of duplicated rows

,Product ID,Description,Price,Rating,Is Prime
888,B0DJPMJ8FS,"YIORYO Headphone Stand, Headphone Holder Stand...",223.17,NaN,0
865,B0DJPMJ8FS,"YIORYO Headphone Stand, Headphone Holder Stand...",223.17,NaN,0
859,B0DJNQTY2Y,"Headset Cable, Headphone Cord,3.5mm Headset Co...",9.28,NaN,0
880,B0DJNQTY2Y,"Headset Cable, Headphone Cord,3.5mm Headset Co...",9.28,NaN,0
933,B0DJMN88TV,MRYFODA Wireless Earbuds Sport Running Ear Hoo...,240.70,NaN,0
...,...,...,...,...,...
186,B00FZLV9G8,Over Ear Wireless Bluetooth Headphones with Mi...,39.95,4.3 out of 5 stars,0
750,B00FZLV9G8,Over Ear Wireless Bluetooth Headphones with Mi...,39.95,4.3 out of 5 stars,0
486,B00FZLV9G8,Over Ear Wireless Bluetooth Headphones with Mi...,39.95,4.3 out of 5 stars,0
1060,B00AF6KOJG,thumbsUp! Zip Earphones - Black,7.99,3.7 out of 5 stars,0


In [153]:
duplicated = df[df.duplicated(keep='first')].sort_values(by = 'Product ID', ascending=False)

In [154]:
# using size to count number of occurrences of each duplicated headphone
duplicated.groupby(['Product ID'])[['Product ID']].size().sort_values(ascending=False)


Product ID
B082P6L3T5    40
B00N3UC444    34
B09PQSVFQT    33
B0992DXFJP    22
B0C6MKMZF1    22
              ..
B0CDGV3W4T     1
B0CDGW1HS2     1
B0CDQ3TWSJ     1
B0CDQ5X8CF     1
B0DJPMJ8FS     1
Length: 206, dtype: int64

In [155]:
# Dropping duplicates
df = df.drop_duplicates()

In [156]:
# Re-checking dataframe after dropping the duplicated rows
df_check(df)


    Data Quality Checks:
    --------------------------------------------
    No. of rows: 1032
    No. of columns: 5
    No. of missing values: 531
    No. of duplicated rows: 0
    No. of duplicated columns: 0
    


### Dealing with Missing Values

In [157]:
df.isna().sum()

Product ID       0
Description      0
Price            0
Rating         531
Is Prime         0
dtype: int64

In [158]:
# Viewing the null value in rating
df[df['Rating'].isna()]

,Product ID,Description,Price,Rating,Is Prime
134,B0DCYYFF3G,"Wireless Headphones for TV Watching, FM Wirele...",29.98,NaN,0
264,B0CBMQKPSY,CAFFIA Wireless Noise Canceling Overhead Headp...,471.96,NaN,0
292,B0D7ZPP1LZ,CN-Outlet Bulk Earbud Headphones 100 Pack for ...,727.86,NaN,0
293,B0D7M7C4M2,"VR Headset, Virtual Reality Headset with Contr...",52.98,NaN,0
295,B09FLTJQWB,NCRD Wireless Noise Canceling Overhead Headpho...,505.30,NaN,0
...,...,...,...,...,...
1495,B0C1XY9DRG,Baluue 5pcs Headphone Anti-lost Rope Strap Wir...,9.69,NaN,0
1496,B0C8FLVZM4,Bluetooth Headphones - Neck Style Wireless Spo...,59.94,NaN,0
1497,B0CD5YX3J5,Fenytay Cat Ear Headphones | Cat Ears Wireless...,14.19,NaN,0
1498,B0D4J6H241,Pink Active Noise Cancelling Headphones Wirele...,16.99,NaN,0


**Comment:**

I have decided to delete rows with null ratings, even though this results in losing about half of my dataset. In this project, my aim is to maintain as much authenticity of the data used in the recommeder system.

Imputing missing rating values with the mean or any other value could introduce some bias and potentially skew recommendations given the large number of missing values.

While reducing the dataset size by 50% seems rather significant, I believe that retaining only authentic and untouched data will improive the reliability of the recommender system. 

Also deciding to srop null rows for price - only 32 rows.

In [159]:
# Dropping all rows with the missing rating 
df = df.dropna()

In [160]:
# Re-checking dataframe
df_check(df)


    Data Quality Checks:
    --------------------------------------------
    No. of rows: 501
    No. of columns: 5
    No. of missing values: 0
    No. of duplicated rows: 0
    No. of duplicated columns: 0
    


### Dealing with Not Specified 

When scraping if information was not present I added a placeholder 'NOT SPECIFIED' I would have to get rid of this in certain columns where I need all info: Price

In [161]:
df.columns       

Index(['Product ID', 'Description', 'Price', 'Rating', 'Is Prime'], dtype='object')

In [162]:
df['Price'].value_counts()

Price
19.99            28
15.99            18
14.99            17
Not Specified    17
24.99            13
                 ..
13.90             1
9.49              1
46.99             1
44.95             1
15.10             1
Name: count, Length: 256, dtype: int64

In [163]:
df = df[df['Price'] != 'Not Specified']

In [164]:
df['Price'].value_counts()

Price
19.99    28
15.99    18
14.99    17
16.99    13
20.99    13
         ..
13.90     1
9.49      1
46.99     1
44.95     1
15.10     1
Name: count, Length: 255, dtype: int64

In [165]:
df['Price'] = df['Price'].str.replace(',', '').astype(float)

In [166]:
df_check(df)


    Data Quality Checks:
    --------------------------------------------
    No. of rows: 484
    No. of columns: 5
    No. of missing values: 0
    No. of duplicated rows: 0
    No. of duplicated columns: 0
    


### Resetting the Index

In [167]:
df = df.reset_index(drop = True)

## Product Description
----

In [168]:
df['Description']

0      OneOdio Wired Over Ear Headphones Hi-Fi Sound ...
1      Lindy NC-60 Wired Active Noise Cancelling (ANC...
2      Sony MDRZX310L.AE Foldable Headphones - Metall...
3      KVIDIO Bluetooth Headphones Over Ear, 65 Hours...
4      Sony MDR-ZX110 Overhead Headphones - Black , B...
                             ...                        
479    3.5mm Earbuds Wired Headphones for Samsung A25...
480    Maeline Bulk Earbuds with 3.5 mm Headphone Plu...
481    On Ear Headphones Bluetooth 5.3 Clip on Headph...
482    TONEMAC H7Pro Hybrid Active Noise Cancelling H...
483    Wired Earphones Headphones WGZBLON Yinyoo FG E...
Name: Description, Length: 484, dtype: object

### Bulk


Seen some cases where prodcuts are not singlular they are often in bulk, true for more pricier items:

In [169]:
df = df[~df['Description'].str.contains('bulk', case=False, na=False)]

In [170]:
df = df.reset_index(drop = True)

In [171]:
df_check(df)


    Data Quality Checks:
    --------------------------------------------
    No. of rows: 431
    No. of columns: 5
    No. of missing values: 0
    No. of duplicated rows: 0
    No. of duplicated columns: 0
    


### Feature Extraction - Wireless

In [172]:
df['Description'] = df['Description'].str.lower()

In [173]:
# using apply args to take in regexp input 
# args expects tuple hence ,
df['Wireless'] = df['Description'].apply(search_description, args=(r'\bwireless\b',))

### Feature Extraction - Noise Cancelling

In [174]:
df['Noise Cancelling'] = df['Description'].apply(search_description, args=(r'\bnoise[-\s]?cancelling\b',))

In [175]:
df['Noise Cancelling'].value_counts()

Noise Cancelling
0    372
1     59
Name: count, dtype: int64

### Feature Extraction - Colour


In [176]:
df['Description']

0      oneodio wired over ear headphones hi-fi sound ...
1      lindy nc-60 wired active noise cancelling (anc...
2      sony mdrzx310l.ae foldable headphones - metall...
3      kvidio bluetooth headphones over ear, 65 hours...
4      sony mdr-zx110 overhead headphones - black , b...
                             ...                        
426    jinserta rgb cat ear headphones,bluetooth 5.3 ...
427    3.5mm earbuds wired headphones for samsung a25...
428    on ear headphones bluetooth 5.3 clip on headph...
429    tonemac h7pro hybrid active noise cancelling h...
430    wired earphones headphones wgzblon yinyoo fg e...
Name: Description, Length: 431, dtype: object

In [177]:
# Apply the function to each row in the Description column
df['Colour'] = df['Description'].apply(get_colour)

In [178]:
df['Colour'].value_counts()

Colour
Not Specified    157
black            103
blue              40
pink              35
green             17
gold              17
white             15
purple            12
red                8
grey               6
beige              4
orange             4
silver             3
yellow             3
gray               2
cyan               2
ivory              1
navy               1
violet             1
Name: count, dtype: int64

In [179]:
df['Colour'] = df['Colour'].replace('gray', 'grey')

In [180]:
df['Colour'] = df['Colour'].replace('gray', 'grey')

In [181]:
df['Colour']  = df['Colour'].replace(['navy','cyan'], 'blue')

In [182]:
df['Colour']  = df['Colour'].replace('violet', 'purple')

In [183]:
df['Colour']  = df['Colour'].replace('ivory', 'white')

### Feature Extraction - Battery Life

In [184]:
df['Battery Life'] = df['Description'].apply(get_battery_life)

### Feature Extraction - Microphone

In [185]:
df['Microphone'] = df['Description'].apply(search_description, args=(r'\b(mic?|microphone?)\b',))

### Feature Extraction: Over Ear

In [186]:
df['Over Ear'] = df['Description'].apply(search_description, args=(r'\b(over[\s-]ear?|overhead?)\b',))

### Feature Extraction : Gaming 

In [187]:
df['Gaming'] = df['Description'].apply(search_description, args=(r'\bgaming\b',))

### Feature Extraction : Foldable 

In [188]:
df['Foldable'] = df['Description'].apply(search_description, args=(r'\bfoldable\b',))

### Feature Extraction : Brand

In [189]:
import spacy

In [190]:
def get_brand(description):
    # Load a pre-trained model
    nlp = spacy.load("en_core_web_sm")

    # Process the text
    doc = nlp(description)
    # Extract named entities
    for ent in doc.ents:
        if ent.label_ == "ORG":  # ORG for organisations/brands
            return ent.text
        else:
            return 'Unknown Brand'

In [191]:
df['Brand'] = df['Description'].apply(get_brand)

In [192]:
df['Brand'].value_counts()

Brand
Unknown Brand                       249
sony                                  8
c8                                    6
rgb                                   4
radio & wired                         3
louise & mann                         3
doqaus bluetooth headphones over      3
betron                                2
mac                                   2
jyps                                  2
konnao kids                           2
android                               2
jyps kids wireless                    2
mp3/4                                 1
ip55                                  1
samsung                               1
samson technologies                   1
koss kph14w                           1
usb                                   1
philips tat8506wt                     1
orzly                                 1
wt/00                                 1
lomiluskr                             1
betron b550                           1
ukcoco                            

-----
**Comment:**

My attempt at using SpaCy to extract the brand names from Product Descrption did not work too well. 

Majority of the data set containes brand names which are unknown and the ones it managed to find - most are not a brand and are common words in description like wireless/bluetooth or a colours. Therefore, I will drop this column from the dataframe.

In [193]:
df = df.drop(columns = ['Brand'], axis = 0)

## Final Clean Up
---

In [194]:
df.head(10)

,Product ID,Description,Price,Rating,Is Prime,Wireless,Noise Cancelling,Colour,Battery Life,Microphone,Over Ear,Gaming,Foldable
0,B07K776ZK6,oneodio wired over ear headphones hi-fi sound ...,34.99,4.4 out of 5 stars,0,0,0,red,Not Specified,0,1,0,0
1,B074DZ39QJ,lindy nc-60 wired active noise cancelling (anc...,58.78,4.2 out of 5 stars,1,0,1,black,Not Specified,0,0,0,0
2,B00I3LUYNG,sony mdrzx310l.ae foldable headphones - metall...,18.00,4.5 out of 5 stars,0,0,0,blue,Not Specified,0,0,0,1
3,B09PQSVFQT,"kvidio bluetooth headphones over ear, 65 hours...",18.99,4.5 out of 5 stars,0,1,0,black,65,1,1,0,1
4,B00NBR70DO,"sony mdr-zx110 overhead headphones - black , b...",14.79,4.5 out of 5 stars,0,0,0,black,Not Specified,0,1,0,0
5,B0C8V45ZF5,roxel rx-90 wired headphones with microphone -...,12.99,4.2 out of 5 stars,0,0,0,black,Not Specified,1,0,0,0
6,B0CQXGBNDV,ailihen wired headphones with microphone and v...,24.99,4.3 out of 5 stars,0,0,0,grey,Not Specified,1,1,0,1
7,B083P1HG9S,lorelei x6 over-ear headphones with microphone...,15.99,4.4 out of 5 stars,0,0,0,black,Not Specified,1,1,0,1
8,B0BCXJYD3G,"bluetooth headphones over-ear, powerlocus wire...",17.99,4.5 out of 5 stars,0,1,0,Not Specified,Not Specified,1,1,0,1
9,B087JWTWF9,artix cl750 wired headphones with mic & volume...,24.78,4.2 out of 5 stars,0,0,1,Not Specified,Not Specified,1,1,0,1


### Checking Duplicated Product IDs

In [195]:
df[df['Product ID'].duplicated(False)].sort_values(by='Product ID')

,Product ID,Description,Price,Rating,Is Prime,Wireless,Noise Cancelling,Colour,Battery Life,Microphone,Over Ear,Gaming,Foldable


In [196]:
df = df.drop_duplicates(subset='Product ID')

----
**Comment:**

Cases where same product ID present with differing features such as prime eligible and not prime eligible. To drop these products as these will cause confusion further down the line.

### Rating

To remove `out of 5 stars` as this information is redundant information.

In [197]:
df['Rating'] = df['Rating'].astype('str')

In [198]:
df['Rating'] = df['Rating'].str.replace('out of 5 stars', '')

In [199]:
df['Rating'] = df['Rating'].astype(float)

### Data Types

In [200]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431 entries, 0 to 430
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Product ID        431 non-null    object 
 1   Description       431 non-null    object 
 2   Price             431 non-null    float64
 3   Rating            431 non-null    float64
 4   Is Prime          431 non-null    int64  
 5   Wireless          431 non-null    int64  
 6   Noise Cancelling  431 non-null    int64  
 7   Colour            431 non-null    object 
 8   Battery Life      431 non-null    object 
 9   Microphone        431 non-null    int64  
 10  Over Ear          431 non-null    int64  
 11  Gaming            431 non-null    int64  
 12  Foldable          431 non-null    int64  
dtypes: float64(2), int64(7), object(4)
memory usage: 43.9+ KB


## Export to CSV
----

In [201]:
df = df.reset_index(drop = True)

In [202]:
cleaned_df = df.copy()

In [203]:
cleaned_df.to_csv('../../data/cleaned_data.csv')

In [204]:
cleaned_df.head(10)

,Product ID,Description,Price,Rating,Is Prime,Wireless,Noise Cancelling,Colour,Battery Life,Microphone,Over Ear,Gaming,Foldable
0,B07K776ZK6,oneodio wired over ear headphones hi-fi sound ...,34.99,4.4,0,0,0,red,Not Specified,0,1,0,0
1,B074DZ39QJ,lindy nc-60 wired active noise cancelling (anc...,58.78,4.2,1,0,1,black,Not Specified,0,0,0,0
2,B00I3LUYNG,sony mdrzx310l.ae foldable headphones - metall...,18.00,4.5,0,0,0,blue,Not Specified,0,0,0,1
3,B09PQSVFQT,"kvidio bluetooth headphones over ear, 65 hours...",18.99,4.5,0,1,0,black,65,1,1,0,1
4,B00NBR70DO,"sony mdr-zx110 overhead headphones - black , b...",14.79,4.5,0,0,0,black,Not Specified,0,1,0,0
5,B0C8V45ZF5,roxel rx-90 wired headphones with microphone -...,12.99,4.2,0,0,0,black,Not Specified,1,0,0,0
6,B0CQXGBNDV,ailihen wired headphones with microphone and v...,24.99,4.3,0,0,0,grey,Not Specified,1,1,0,1
7,B083P1HG9S,lorelei x6 over-ear headphones with microphone...,15.99,4.4,0,0,0,black,Not Specified,1,1,0,1
8,B0BCXJYD3G,"bluetooth headphones over-ear, powerlocus wire...",17.99,4.5,0,1,0,Not Specified,Not Specified,1,1,0,1
9,B087JWTWF9,artix cl750 wired headphones with mic & volume...,24.78,4.2,0,0,1,Not Specified,Not Specified,1,1,0,1


In [205]:
df_check(cleaned_df)


    Data Quality Checks:
    --------------------------------------------
    No. of rows: 431
    No. of columns: 13
    No. of missing values: 0
    No. of duplicated rows: 0
    No. of duplicated columns: 0
    


## Conclusion
-------



In this notebook, I have cleaned the dataset ready for EDA. Here is a review of the key steps/insights:

1. **Dropped Duplicates:** Removed duplicate entries to maintain data quality

2. **Addressed Missing Values:** Dropped all records with null values in ratings column to keep authenticity of the data. 

3. **Feature Extraction:** Performed feature extraction to enhance the dataset, focusing on creating relevant features from product descriptions.

With the dataset now cleaned and features added, I am prepared to proceed to the next phase: EDA. 
